In [1]:
!pip install -q polars==0.16.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 43.6 MB/s eta 0:00:00


In [2]:
VER = 2

In [3]:
import pandas as pd, numpy as np
import pickle, glob, gc
from collections import Counter
import itertools
# multiprocessing 
import psutil
N_CORES = psutil.cpu_count()     # Available CPU cores
print(f"N Cores : {N_CORES}")
from multiprocessing import Pool

N Cores : 8


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
paths = ['/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/submission/handcrafted6.pqt',
         '/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/submission/handcrafted7.pqt',
         '/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/submission/handcrafted21.pqt',
         '/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/submission/handcrafted25.pqt']

In [6]:
import polars as pl
def read_sub(path, weight=1): # by default let us assing the weight of 1 to predictions from each submission, this will be akin to a standard vote ensemble
    '''a helper function for loading and preprocessing submissions'''
    return (
        pl.read_parquet(path)
            .with_column(pl.col('labels').str.split(by=' '))
            .with_column(pl.lit(weight).alias('vote'))
            .explode('labels')
            .rename({'labels': 'aid'})
            .with_column(pl.col('aid').cast(pl.UInt32)) # we are casting the `aids` to `Int32`! memory management is super important to ensure we don't run out of resources
            .with_column(pl.col('vote').cast(pl.UInt8))
    )

In [7]:
subs = [read_sub(path) for path in paths]

<ipython-input-6-19365d2ba5f6>:5: DeprecationWarning: `with_column` has been deprecated in favor of `with_columns`. This method will be removed in version 0.17.0
  pl.read_parquet(path)


In [8]:
subs = subs[0].join(subs[1], how='outer', on=['session_type', 'aid']).join(subs[2], how='outer', on=['session_type', 'aid'], suffix='_right2').join(subs[3], how='outer', on=['session_type', 'aid'], suffix='_right3')
subs.head()

session_type,aid,vote,vote_right,vote_right2,vote_right3
str,u32,u8,u8,u8,u8
"""12899779_click...",59625,1,1,1,1
"""12899779_click...",1253524,1,1,1,1
"""12899779_click...",737445,1,1,1,1
"""12899779_click...",438191,1,1,1,1
"""12899779_click...",731692,1,1,1,1


In [9]:
subs = (subs
    .fill_null(0)
    .with_column((pl.col('vote') + pl.col('vote_right') + pl.col('vote_right2') + pl.col('vote_right3')).alias('vote_sum'))
    .drop(['vote', 'vote_right','vote_right2','vote_right3'])
    .sort(by='vote_sum')
    .reverse()
)

<ipython-input-9-71a6a13f7612>:1: DeprecationWarning: `with_column` has been deprecated in favor of `with_columns`. This method will be removed in version 0.17.0
  subs = (subs


In [10]:
preds = subs.groupby('session_type').agg([
    pl.col('aid').head(20).alias('labels')
])

preds = preds.with_column(pl.col('labels').apply(lambda lst: ' '.join([str(aid) for aid in lst])))

<ipython-input-10-a40be3906ad3>:5: DeprecationWarning: `with_column` has been deprecated in favor of `with_columns`. This method will be removed in version 0.17.0
  preds = preds.with_column(pl.col('labels').apply(lambda lst: ' '.join([str(aid) for aid in lst])))


In [11]:
%%time
preds.write_csv(f'ensemble{VER}.csv')
preds.write_csv(f'/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/submission/ensemble{VER}.csv')

CPU times: user 4.38 s, sys: 2.41 s, total: 6.79 s
Wall time: 5.31 s


In [12]:
!pip install kaggle -q
import os
import json
f = open("/content/drive/MyDrive/Colab Notebooks/kaggle/kaggle.json", 'r')
json_data = json.load(f)
os.environ['KAGGLE_USERNAME'] = json_data['username']
os.environ['KAGGLE_KEY'] = json_data['key']

In [13]:
!kaggle competitions submit -c otto-recommender-system -f ensemble2.csv -m ""

100% 780M/780M [00:30<00:00, 26.6MB/s]
Successfully submitted to OTTO – Multi-Objective Recommender System